In [ ]:
! pip install transformers

In [ ]:
import pandas as pd
data = pd.read_csv("../utterances.csv")
data.columns = data.columns.str.lower()

for col in data.select_dtypes(include="object"):
    data[col] = data[col].str.lower()

data.drop(columns=['unnamed: 0'], errors='ignore', inplace=True)
data

,transcript_id,domain,intent,reason_for_call,short_intent,speaker,text,turn_no
0,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,agent,"hello, thank you for contacting buynow. this i...",1
1,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,customer,"hello, i'm calling about an order that shows d...",2
2,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,agent,i'm sorry to hear that. i'll definitely help y...,3
3,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,customer,it's 9595912. the tracking was marked delivere...,4
4,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,agent,"let me pull that up right away. okay, i see th...",5
...,...,...,...,...,...,...,...,...
84460,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",customer,"yes, that's perfect. and i need to switch my p...",8
84461,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",agent,i can process both changes. the plan change ta...,9
84462,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",customer,[provides card ending in 2950],10
84463,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",agent,updated. you'll save $33/month starting next c...,11


In [ ]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: cardiffnlp/twitter-roberta-base-sentiment
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
LABEL_TO_SCORE = {
    "LABEL_0": -1.0,
    "LABEL_1": 0.0,
    "LABEL_2": 1.0
}

def get_sentiment(text):
    if not isinstance(text, str) or not text.strip():
        return 0.0

    result = sentiment_model(text[:512])[0]  # truncate safely
    return LABEL_TO_SCORE[result["label"]] * result["score"]


In [ ]:
len(data)

84465

In [ ]:
BATCH_SIZE = 1000
START_ROW = 0
END_ROW = 84464


for start in range(START_ROW, min(END_ROW, len(data)), BATCH_SIZE):
    end = start + BATCH_SIZE

    print(f"Processing rows {start} to {min(end, END_ROW) - 1}")

    data.loc[start:min(end, END_ROW)-1, "sentiment"] = (
        data.loc[start:min(end, END_ROW)-1, "text"]
        .apply(get_sentiment)
    )



Processing rows 0 to 999
Processing rows 1000 to 1999
Processing rows 2000 to 2999
Processing rows 3000 to 3999
Processing rows 4000 to 4999
Processing rows 5000 to 5999
Processing rows 6000 to 6999
Processing rows 7000 to 7999
Processing rows 8000 to 8999
Processing rows 9000 to 9999
Processing rows 10000 to 10999
Processing rows 11000 to 11999
Processing rows 12000 to 12999
Processing rows 13000 to 13999
Processing rows 14000 to 14999
Processing rows 15000 to 15999
Processing rows 16000 to 16999
Processing rows 17000 to 17999
Processing rows 18000 to 18999
Processing rows 19000 to 19999
Processing rows 20000 to 20999
Processing rows 21000 to 21999
Processing rows 22000 to 22999
Processing rows 23000 to 23999
Processing rows 24000 to 24999
Processing rows 25000 to 25999
Processing rows 26000 to 26999
Processing rows 27000 to 27999
Processing rows 28000 to 28999
Processing rows 29000 to 29999
Processing rows 30000 to 30999
Processing rows 31000 to 31999
Processing rows 32000 to 32999
P

In [ ]:
data.to_csv("utterances_final.csv", index=False)

In [3]:
import pandas as pd

dataset = pd.read_csv("../Codebase/utterances_final.csv")

In [4]:
CRITICAL_PATTERNS = {
    # ============= CUSTOMER PATTERNS =============

    # FRUSTRATION & EMOTION
    "extreme_frustration": [
        "extremely frustrated", "very angry", "fed up", "had enough",
        "this is ridiculous", "this is unacceptable", "worst service",
        "beyond frustrated", "incredibly upset", "absolutely furious",
        "losing my patience", "sick and tired", "can't believe this",
        "outrageous", "disgusted", "appalled"
    ],

    "moderate_frustration": [
        "frustrated", "annoyed", "irritated", "disappointed",
        "not happy", "upset", "concerned", "worried",
        "bothered", "inconvenient", "troublesome"
    ],

    "anger_expression": [
        "angry", "mad", "infuriated", "enraged", "livid",
        "furious", "irate", "incensed", "outraged"
    ],

    # ESCALATION INDICATORS
    "escalation_request": [
        "speak to supervisor", "talk to manager", "escalate this",
        "someone in charge", "speak to your boss", "manager now",
        "corporate office", "higher authority", "senior representative",
        "complaint department", "want to escalate", "transfer me up",
        "need to speak with", "get me your manager"
    ],

    "escalation_threat": [
        "take this higher", "go above you", "contact corporate",
        "file a complaint", "report this", "take legal action",
        "contact the BBB", "social media", "leave a review",
        "lawyer", "sue", "lawsuit", "attorney general"
    ],

    # REPEAT ISSUES
    "repeat_contact": [
        "called multiple times", "calling again", "third time calling",
        "many times", "keep calling", "repeatedly contacted",
        "called last week", "spoke to someone yesterday",
        "this is my second call", "fourth representative",
        "already talked to", "keep having to call",
        "how many times", "every time I call"
    ],

    "ongoing_issue": [
        "still not fixed", "still happening", "same problem",
        "continues to", "ongoing issue", "persistent problem",
        "hasn't been resolved", "not working yet", "still broken",
        "for weeks now", "for months", "still waiting"
    ],

    # BROKEN PROMISES
    "broken_promise": [
        "you promised", "was told", "said it would be", "assured me",
        "guaranteed", "never happened", "didn't receive",
        "no callback", "never called back", "no follow-up",
        "supposed to", "should have been", "expected to",
        "committed to", "pledged", "never delivered"
    ],

    "unmet_expectations": [
        "expected better", "thought it would", "should have",
        "normally this", "used to be", "advertised as",
        "not what I paid for", "not as described"
    ],

    # URGENCY & IMPACT
    "high_urgency": [
        "need this now", "urgent", "emergency", "immediately",
        "right away", "as soon as possible", "can't wait",
        "time sensitive", "critical", "pressing matter",
        "today", "by end of day"
    ],

    "severe_impact": [
        "can't work", "lost money", "financial loss", "missed deadline",
        "affecting my job", "business impact", "can't pay bills",
        "eviction", "losing customers", "stuck", "stranded",
        "no access to", "completely down", "out of service"
    ],

    # LOYALTY & RELATIONSHIP
    "loyalty_reference": [
        "long time customer", "been with you for years", "loyal customer",
        "always used your service", "never had problems before",
        "faithful customer", "decade", "years of service",
        "premium member", "paying customer"
    ],

    "threatening_to_leave": [
        "cancel my account", "close my account", "switching to",
        "take my business elsewhere", "find another provider",
        "done with you", "leaving", "terminating service",
        "competitor", "alternative", "cancellation"
    ],

    # COMPLAINT INTENT
    "complaint_intent": [
        "file a complaint", "formal complaint", "make a complaint",
        "lodge a complaint", "complain about", "report this",
        "submit a complaint", "grievance", "dispute"
    ],

    "negative_review_threat": [
        "write a review", "post online", "tell everyone",
        "social media", "Twitter", "Facebook", "Yelp",
        "Google review", "warn others", "spread the word",
        "bad publicity", "public complaint"
    ],

    # DISTRUST & CREDIBILITY
    "loss_of_trust": [
        "don't trust", "can't believe", "doubt", "skeptical",
        "not confident", "no faith", "questionable",
        "unreliable", "not credible", "suspicious"
    ],

    "questioning_competence": [
        "do you even know", "are you sure", "seems like you don't",
        "don't think you can", "incapable", "not qualified",
        "doesn't seem like", "clearly don't understand",
        "incompetent", "untrained"
    ],

    # FINANCIAL CONCERNS
    "financial_dispute": [
        "overcharged", "wrong amount", "charged twice",
        "unauthorized charge", "billing error", "refund",
        "money back", "credit", "reimbursement",
        "fraudulent", "incorrect bill", "dispute charge"
    ],

    "compensation_request": [
        "want compensation", "deserve refund", "credit my account",
        "discount", "waive the fee", "free month",
        "make this right", "compensate me", "reimburse"
    ],

    # ============= AGENT PATTERNS =============

    # EMPATHY & ACKNOWLEDGMENT
    "empathy": [
        "I'm sorry", "I apologize", "I understand", "I can see",
        "that must be frustrating", "I hear you", "I appreciate",
        "thank you for your patience", "I recognize",
        "totally understand", "completely understand"
    ],

    "validation": [
        "you're right", "that's valid", "I agree", "absolutely",
        "that makes sense", "I see your point", "fair concern",
        "understandable", "reasonable"
    ],

    # AGENT LIMITATIONS
    "agent_limitation": [
        "I cannot", "I'm unable to", "I don't have access",
        "not authorized", "can't help with", "beyond my scope",
        "outside my authority", "unable to assist",
        "don't have permission", "restricted from",
        "not available to me", "system won't allow"
    ],

    "policy_citation": [
        "company policy", "our policy states", "according to policy",
        "policy requires", "per our terms", "terms and conditions",
        "contractual agreement", "standard procedure",
        "regulatory requirement", "compliance"
    ],

    "technical_limitation": [
        "system limitation", "technical issue", "system doesn't allow",
        "not supported", "platform restriction", "backend limitation",
        "database constraint", "technical constraint"
    ],

    # SOLUTIONS & ACTIONS
    "solution_offered": [
        "I can help", "let me", "I will", "I'll get this",
        "here's what I can do", "I'm going to", "I'll make sure",
        "allow me to", "I'm able to", "I can assist",
        "let me take care", "I'll handle"
    ],

    "investigation": [
        "let me check", "looking into", "investigating",
        "reviewing your account", "pulling up", "researching",
        "examining", "verifying", "confirming"
    ],

    "workaround_offered": [
        "alternative solution", "another option", "workaround",
        "in the meantime", "temporary solution", "different approach",
        "can try", "alternative method"
    ],

    # ESCALATION & TRANSFER
    "escalation_trigger": [
        "transfer to supervisor", "escalate to", "manager",
        "senior representative", "specialist", "supervisor",
        "higher level support", "tier 2", "advanced support",
        "team lead", "department head"
    ],

    "transfer_intent": [
        "transfer you to", "connect you with", "route you to",
        "forward to", "send to", "direct you to",
        "put you through to"
    ],

    # DELAYS & WAIT TIMES
    "delay_notification": [
        "will take", "processing time", "business days",
        "up to", "typically takes", "estimated time",
        "may take longer", "could be delayed", "waiting period",
        "turnaround time"
    ],

    "hold_request": [
        "please hold", "one moment", "bear with me",
        "just a second", "checking now", "pulling that up",
        "give me a moment", "brief hold"
    ],

    # INFORMATION GATHERING
    "information_gathering": [
        "can you provide", "may I have", "what is your",
        "can you confirm", "please verify", "need to know",
        "could you tell me", "help me understand",
        "can you describe", "what's your"
    ],

    "clarification": [
        "just to clarify", "want to make sure", "confirm that",
        "so you're saying", "if I understand correctly",
        "let me verify", "to be clear", "double checking"
    ],

    # PROMISES & COMMITMENTS
    "commitment_made": [
        "I promise", "I guarantee", "I assure you",
        "definitely will", "absolutely will", "you have my word",
        "will make sure", "committed to", "will ensure"
    ],

    "timeline_given": [
        "by end of day", "within 24 hours", "in 3-5 business days",
        "by tomorrow", "within the hour", "next week",
        "by Friday", "specific date", "exact timeframe"
    ],

    # DEFENSIVE PATTERNS
    "deflection": [
        "that's not our", "different department", "not my area",
        "that's handled by", "outside my scope", "third party",
        "vendor issue", "not our responsibility",
        "carrier problem", "manufacturer's fault"
    ],

    "minimization": [
        "it's just", "only a", "minor issue", "small problem",
        "shouldn't take long", "simple fix", "easy solution",
        "not a big deal", "happens sometimes"
    ],

    # POSITIVE RESOLUTION
    "resolution_confirmation": [
        "resolved", "fixed", "taken care of", "all set",
        "completed", "processed", "finalized",
        "issue is closed", "matter is settled"
    ],

    "follow_up_commitment": [
        "will follow up", "will call you back", "will email you",
        "will send confirmation", "will update you",
        "keep you posted", "notify you when"
    ],

    # ============= OUTCOME PATTERNS =============

    "successful_resolution": [
        "thank you for your help", "that works", "perfect",
        "great", "appreciate it", "satisfied",
        "problem solved", "all good now"
    ],

    "partial_resolution": [
        "I guess that works", "okay for now", "better than nothing",
        "will try that", "see how it goes", "hope this works"
    ],

    "unresolved_frustration": [
        "this doesn't help", "still not solved", "doesn't fix",
        "not good enough", "unacceptable solution",
        "that won't work", "same problem", "didn't resolve"
    ]
}


SENTIMENT_PATTERNS = {
    "very_positive": [
        "excellent", "outstanding", "wonderful", "fantastic",
        "amazing", "great job", "very helpful", "exceeded expectations",
        "perfect", "couldn't be better", "highly satisfied"
    ],

    "positive": [
        "good", "helpful", "nice", "appreciate", "thank you",
        "thanks", "glad", "happy", "satisfied", "pleased"
    ],

    "neutral": [
        "okay", "fine", "alright", "understand", "got it",
        "I see", "makes sense", "clear"
    ],

    "negative": [
        "not good", "bad", "poor", "disappointing", "unhappy",
        "dissatisfied", "not pleased", "unfortunate", "subpar"
    ],

    "very_negative": [
        "terrible", "awful", "horrible", "worst", "unacceptable",
        "outrageous", "appalling", "disgusting", "pathetic",
        "abysmal", "atrocious"
    ]
}


CONTEXTUAL_PATTERNS = {
    "first_contact": [
        "first time calling", "never contacted before",
        "new to this", "first issue", "initial contact"
    ],

    "existing_ticket": [
        "case number", "ticket number", "reference number",
        "previous call", "open ticket", "existing case"
    ],

    "time_pressure": [
        "leaving soon", "flight in", "meeting at",
        "deadline", "time sensitive", "running out of time",
        "need by", "due date"
    ],

    "vulnerable_customer": [
        "elderly", "disabled", "limited mobility",
        "don't understand technology", "not tech savvy",
        "medical condition", "need assistance"
    ]
}

In [5]:
def detect_pattern_groups(text, pattern_dict):
    if not isinstance(text, str):
        return []

    text_lower = text.lower()
    matched = []

    for pattern_name, keywords in pattern_dict.items():
        if any(keyword in text_lower for keyword in keywords):
            matched.append(pattern_name)

    return matched


In [6]:
dataset["critical_patterns"] = None
dataset["critical_patterns"] = dataset["text"].apply(
    lambda x: detect_pattern_groups(x, CRITICAL_PATTERNS)
)



In [7]:
dataset.columns

Index(['transcript_id', 'domain', 'intent', 'reason_for_call', 'short_intent',
       'speaker', 'text', 'turn_no', 'sentiment', 'critical_patterns'],
      dtype='object')

In [8]:
sentiment_counts = {
    "negative": (dataset["sentiment"] < 0).sum(),
    "neutral": (dataset["sentiment"] == 0).sum(),
    "positive": (dataset["sentiment"] > 0).sum()
}

In [9]:
sentiment_counts

{'negative': np.int64(21786),
 'neutral': np.int64(42903),
 'positive': np.int64(19776)}

In [ ]:
dataset

,transcript_id,domain,intent,reason_for_call,short_intent,speaker,text,turn_no,sentiment,critical_patterns
0,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,agent,"hello, thank you for contacting buynow. this i...",1,0.613491,[]
1,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,customer,"hello, i'm calling about an order that shows d...",2,-0.895869,[]
2,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,agent,i'm sorry to hear that. i'll definitely help y...,3,0.000000,[]
3,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,customer,it's 9595912. the tracking was marked delivere...,4,0.000000,[]
4,6794-8660-4606-3216,e-commerce & retail,delivery investigation,customer james bailey reported a smart watch s...,delivery investigation,agent,"let me pull that up right away. okay, i see th...",5,0.000000,"[high_urgency, solution_offered]"
...,...,...,...,...,...,...,...,...,...,...
84460,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",customer,"yes, that's perfect. and i need to switch my p...",8,0.917225,"[financial_dispute, successful_resolution]"
84461,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",agent,i can process both changes. the plan change ta...,9,0.000000,[]
84462,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",customer,[provides card ending in 2950],10,0.000000,[]
84463,5013-5906-1491-6772,technology support,"multiple issues - technical, plan & payment",customer margaret williams reported wifi cover...,"multiple issues - technical, plan & payment",agent,updated. you'll save $33/month starting next c...,11,0.000000,[]


In [ ]:
import json
import pandas as pd
import ast

def create_conversation_summaries(df):
    """Create conversation-level summaries"""

    summaries = []

    for transcript_id in df['transcript_id'].unique():
        conv_df = df[df['transcript_id'] == transcript_id].sort_values('turn_no')

        # Extract metadata
        metadata = {
            'transcript_id': transcript_id,
            'domain': conv_df.iloc[0]['domain'],
            'intent': conv_df.iloc[0]['intent'],
            'reason_for_call': conv_df.iloc[0]['reason_for_call'],
            'short_intent': conv_df.iloc[0]['short_intent'],
            'total_turns': len(conv_df)
        }

        # Collect critical turns
        critical_turns = []
        all_patterns = []

        for _, row in conv_df.iterrows():
            # Fix: Convert to string first, then check
            patterns_str = str(row['critical_patterns'])

            # Check if not empty or '[]'
            if patterns_str and patterns_str != '[]' and patterns_str != 'nan':
                try:
                    # Try parsing as JSON
                    patterns = json.loads(patterns_str)

                    # Handle different data types
                    if isinstance(patterns, list) and len(patterns) > 0:
                        critical_turns.append({
                            'turn_no': row['turn_no'],
                            'speaker': row['speaker'],
                            'text': row['text'],
                            'patterns': patterns
                        })
                        all_patterns.extend(patterns)

                except (json.JSONDecodeError, TypeError):
                    # If JSON parsing fails, try ast.literal_eval
                    try:
                        patterns = ast.literal_eval(patterns_str)
                        if isinstance(patterns, list) and len(patterns) > 0:
                            critical_turns.append({
                                'turn_no': row['turn_no'],
                                'speaker': row['speaker'],
                                'text': row['text'],
                                'patterns': patterns
                            })
                            all_patterns.extend(patterns)
                    except:
                        pass  # Skip malformed data

        # Calculate sentiment trajectory
        sentiments = conv_df['sentiment'].tolist()
        sentiment_trajectory = 'stable'
        if len(sentiments) > 1:
            if sentiments[-1] < sentiments[0] - 0.3:
                sentiment_trajectory = 'declining'
            elif sentiments[-1] > sentiments[0] + 0.3:
                sentiment_trajectory = 'improving'

        # Determine outcome
        outcome = determine_outcome(all_patterns, sentiment_trajectory)

        summaries.append({
            **metadata,
            'critical_turn_count': len(critical_turns),
            'critical_turn_numbers': [t['turn_no'] for t in critical_turns],
            'unique_patterns': list(set(all_patterns)),
            'pattern_count': len(all_patterns),
            'avg_sentiment': round(conv_df['sentiment'].mean(), 3),
            'min_sentiment': round(conv_df['sentiment'].min(), 3),
            'sentiment_trajectory': sentiment_trajectory,
            'outcome': outcome
        })

    return pd.DataFrame(summaries)

def determine_outcome(patterns, sentiment_trajectory):
    """Determine conversation outcome"""

    escalation_patterns = [
        'escalation_request', 'escalation_threat',
        'complaint_intent', 'extreme_frustration'
    ]

    resolution_patterns = [
        'successful_resolution', 'solution_offered'
    ]

    if any(p in patterns for p in escalation_patterns):
        return 'escalation'
    elif sentiment_trajectory == 'improving' or any(p in patterns for p in resolution_patterns):
        return 'resolved'
    elif sentiment_trajectory == 'declining':
        return 'unresolved'
    else:
        return 'in_progress'

# Usage
conv_summary_df = create_conversation_summaries(dataset)
conv_summary_df.to_csv('conversation_level_summary.csv', index=False)

In [10]:
import pandas as pd
import json
import ast

# 1. Load the raw datasets
turn_df = pd.read_csv("../Codebase/utterances_final.csv")
conv_summary_df = pd.read_csv("../Codebase/conversation_level_summary.csv")

# 2. Helper function to safely parse string-arrays into Python lists
def safe_parse_list(x):
    if pd.isna(x): return []
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return []

# 3. Merging: Bring conversation-level context into every turn
# This allows each turn to 'know' the final Outcome and Critical Turns list
enriched_df = pd.merge(
    turn_df,
    conv_summary_df[['transcript_id', 'outcome', 'critical_turn_numbers', 'unique_patterns', 'reason_for_call']],
    on='transcript_id',
    how='inner'
)

# 4. Cleaning & Logic Application
print("Cleaning and applying evidence logic...")
enriched_df['critical_turn_numbers'] = enriched_df['critical_turn_numbers'].apply(safe_parse_list)

def is_causal_evidence(row):
    # Condition 1: Turn is explicitly marked as critical in summary
    is_critical = row['turn_no'] in row['critical_turn_numbers']

    # Condition 2: Turn has negative sentiment (Your new requirement)
    try:
        sentiment = float(row['sentiment'])
    except:
        sentiment = 0.0
    is_negative = sentiment < 0.0

    return is_critical or is_negative

# Filter the 80k rows down to high-value evidence turns
evidence_df = enriched_df[enriched_df.apply(is_causal_evidence, axis=1)].copy()

print(f"✅ Data Cleaned: {len(turn_df)} turns reduced to {len(evidence_df)} high-value evidence turns.")

Cleaning and applying evidence logic...
✅ Data Cleaned: 84465 turns reduced to 51697 high-value evidence turns.


In [11]:
sentiment_counts2 = {
    "negative": (evidence_df["sentiment"] < 0).sum(),
    "neutral": (evidence_df["sentiment"] == 0).sum(),
    "positive": (evidence_df["sentiment"] > 0).sum()
}

#sentiment_counts2

In [12]:
sentiment_counts2

{'negative': np.int64(21786),
 'neutral': np.int64(20596),
 'positive': np.int64(9315)}

In [13]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    F

In [ ]:
import shutil
import os

# Define the paths
paths = ["./chroma_db_turns", "./chroma_db_conversations"]

for path in paths:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"🗑️ Deleted: {path}")

print("✨ Disk is 100% clean. No more configuration conflicts.")

✨ Disk is 100% clean. No more configuration conflicts.


In [14]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions  # UPDATE 1: Needed for dimension fix
import json
import ast
import os
import shutil  # UPDATE 2: Needed for hard reset

# 1. SETUP PATHS
PATH_TURNS = "../Codebase/chroma_db_turns3"

# UPDATE 3: HARD RESET (Prevents "Embedding function conflict" error)
if os.path.exists(PATH_TURNS):
    shutil.rmtree(PATH_TURNS)
    print("🧹 Cleaned old database to prevent conflicts.")

# UPDATE 4: DEFINE EMBEDDING FUNCTION (Fixes 768 vs 384 dimension error)
# This tells ChromaDB exactly which model to use so the math matches.
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="multi-qa-mpnet-base-dot-v1",
    device="cuda"  # Enables T4 GPU
)

# 2. ENRICH DATA
enriched_df = pd.merge(
    turn_df,
    conv_summary_df[['transcript_id', 'outcome', 'critical_turn_numbers', 'unique_patterns']],
    on='transcript_id',
    how='left'
)

# 3. INITIALIZE CHROMA
client_turn = chromadb.PersistentClient(path=PATH_TURNS)

# UPDATE 5: BIND FUNCTION TO COLLECTION
col_turn = client_turn.get_or_create_collection(
    name="evidence_turns",
    embedding_function=embedding_func,  # <--- CRITICAL FIX: Links model to DB
    metadata={"hnsw:space": "ip"}
)

def safe_parse(data):
    if pd.isna(data): return []
    try: return ast.literal_eval(data) if isinstance(data, str) else data
    except: return []

# 4. PREPARE DOCUMENTS
turn_docs, turn_ids, turn_metas = [], [], []

print("🔨 Processing individual turns for evidence indexing...")
for _, row in enriched_df.iterrows():
    critical_nums = safe_parse(row['critical_turn_numbers'])

    # UPDATE 6: IMPROVED LOGIC (Critical List OR Negative Sentiment)
    try: sentiment = float(row['sentiment'])
    except: sentiment = 0.0

    is_critical = row['turn_no'] in critical_nums
    is_negative = sentiment < 0.0

    # Keep if EITHER is true (The "Safety Net" for high Recall)
    if is_critical or is_negative:
        doc = f"Outcome: {row['outcome']} | Speaker: {row['speaker']} | Text: {row['text']}"

        turn_docs.append(doc)
        turn_ids.append(f"{row['transcript_id']}_{row['turn_no']}")
        turn_metas.append({
            "transcript_id": str(row['transcript_id']),
            "turn_no": int(row['turn_no']),
            "outcome": str(row['outcome']),
            "patterns": str(row['unique_patterns']),
            "sentiment": sentiment
        })

# 5. BATCH ADD
batch_size = 5000 # Increased size since we are using GPU
print(f"🚀 Indexing {len(turn_docs)} turns...")

for i in range(0, len(turn_docs), batch_size):
    col_turn.add(
        documents=turn_docs[i:i+batch_size],
        ids=turn_ids[i:i+batch_size],
        metadatas=turn_metas[i:i+batch_size]
    )
    print(f"  ➜ Indexed {min(i+batch_size, len(turn_docs))}/{len(turn_docs)}")

print(f"✅ Successfully indexed {len(turn_ids)} Evidence Turns at {PATH_TURNS}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔨 Processing individual turns for evidence indexing...
🚀 Indexing 51697 turns...
  ➜ Indexed 5000/51697
  ➜ Indexed 10000/51697
  ➜ Indexed 15000/51697
  ➜ Indexed 20000/51697
  ➜ Indexed 25000/51697
  ➜ Indexed 30000/51697
  ➜ Indexed 35000/51697
  ➜ Indexed 40000/51697
  ➜ Indexed 45000/51697
  ➜ Indexed 50000/51697
  ➜ Indexed 51697/51697
✅ Successfully indexed 51697 Evidence Turns at ./chroma_db_turns3


In [15]:
import shutil

# 1. Zip the folder
# This creates a file named 'chroma_db_submission.zip'
shutil.make_archive('chroma_db_submission', 'zip', '../Codebase/chroma_db_turns3')

# 2. Download to your computer
from google.colab import files
files.download('chroma_db_submission.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
import json
import ast

# Load your datasets
turn_df = pd.read_csv("../Codebase/utterances_final.csv")
conv_summary_df = pd.read_csv("../Codebase/conversation_level_summary.csv")

# Initialize embedding model - optimized for dot product
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

def safe_parse_patterns(pattern_data):
    """Safely parse pattern data without using eval()"""
    if not isinstance(pattern_data, str):
        return pattern_data if isinstance(pattern_data, list) else []
    try:
        # Use ast.literal_eval instead of eval() for safety
        return ast.literal_eval(pattern_data)
    except:
        return []

def safe_parse_critical_turns(critical_turn_data):
    """Safely parse critical turn numbers"""
    if pd.isna(critical_turn_data):
        return []
    if not isinstance(critical_turn_data, str):
        return critical_turn_data if isinstance(critical_turn_data, list) else []
    try:
        return ast.literal_eval(critical_turn_data)
    except:
        return []

def create_conversation_documents(turn_df, conv_summary_df):
    """Create rich documents for embedding with evidence spans"""
    documents = []
    ids = []
    metadatas = []

    for _, conv_row in conv_summary_df.iterrows():
        transcript_id = conv_row['transcript_id']

        # Get all turns for this conversation
        conv_turns = turn_df[turn_df['transcript_id'] == transcript_id].sort_values('turn_no')

        # Create full conversation text
        full_text = ' '.join(conv_turns['text'].tolist())

        # Safely parse patterns
        unique_patterns = safe_parse_patterns(conv_row['unique_patterns'])
        patterns_str = ', '.join(unique_patterns) if unique_patterns else 'None'

        # Safely parse critical turn numbers for evidence spans
        critical_turn_numbers = safe_parse_critical_turns(conv_row.get('critical_turn_numbers', []))

        # Extract critical turns text for better semantic search
        critical_turns_text = ""
        if critical_turn_numbers:
            critical_turns = conv_turns[conv_turns['turn_no'].isin(critical_turn_numbers)]
            critical_turns_text = ' '.join(critical_turns['text'].tolist())

        # Create rich searchable document with emphasis on critical evidence
        doc = f"""
        Domain: {conv_row['domain']}
        Intent: {conv_row['intent']}
        Outcome: {conv_row['outcome']}
        Issue: {conv_row['reason_for_call']}

        Critical Patterns: {patterns_str}

        Sentiment Trajectory: {conv_row['sentiment_trajectory']}
        Average Sentiment: {conv_row['avg_sentiment']}

        Critical Evidence Turns:
        {critical_turns_text if critical_turns_text else 'No critical turns identified'}

        Full Conversation:
        {full_text}
        """

        documents.append(doc)
        ids.append(transcript_id)

        # Store critical turn indices in metadata for fast evidence retrieval
        metadatas.append({
            'domain': str(conv_row['domain']),
            'intent': str(conv_row['intent']),
            'outcome': str(conv_row['outcome']),
            'sentiment_trajectory': str(conv_row['sentiment_trajectory']),
            'critical_turn_count': int(conv_row['critical_turn_count']),
            'avg_sentiment': float(conv_row['avg_sentiment']),
            'critical_turn_numbers': json.dumps(critical_turn_numbers),  # Store as JSON string
            'unique_patterns': json.dumps(unique_patterns)  # Store as JSON string
        })

    return documents, ids, metadatas

# Create documents
print("Creating conversation documents...")
print("Using model: multi-qa-mpnet-base-dot-v1")
print("Distance metric: Inner Product (Dot Product)")
documents, ids, metadatas = create_conversation_documents(turn_df, conv_summary_df)

# Delete existing collection if it exists (to ensure clean indexing with new settings)
try:
    chroma_client.delete_collection(name="conversations")
    print("✓ Deleted existing collection")
except:
    pass

# Create ChromaDB collection with Inner Product (Dot Product) distance
collection = chroma_client.create_collection(
    name="conversations",
    metadata={"hnsw:space": "ip"}  # Inner Product for multi-qa-mpnet-base-dot-v1
)

# Add to vector DB
print("Adding to vector database...")
batch_size = 100
for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    batch_metas = metadatas[i:i+batch_size]

    collection.add(
        documents=batch_docs,
        ids=batch_ids,
        metadatas=batch_metas
    )
    print(f"  Indexed {min(i+batch_size, len(documents))}/{len(documents)} conversations")

print(f"\n✅ Successfully indexed {len(documents)} conversations")
print(f"✅ Using Inner Product similarity (optimized for multi-qa-mpnet-base-dot-v1)")
print(f"✅ Critical turn numbers stored in metadata for evidence retrieval")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating conversation documents...
Using model: multi-qa-mpnet-base-dot-v1
Distance metric: Inner Product (Dot Product)
Adding to vector database...


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:38<00:00, 844kiB/s]


  Indexed 100/5037 conversations
  Indexed 200/5037 conversations
  Indexed 300/5037 conversations
  Indexed 400/5037 conversations
  Indexed 500/5037 conversations
  Indexed 600/5037 conversations
  Indexed 700/5037 conversations
  Indexed 800/5037 conversations
  Indexed 900/5037 conversations
  Indexed 1000/5037 conversations
  Indexed 1100/5037 conversations
  Indexed 1200/5037 conversations
  Indexed 1300/5037 conversations
  Indexed 1400/5037 conversations
  Indexed 1500/5037 conversations
  Indexed 1600/5037 conversations
  Indexed 1700/5037 conversations
  Indexed 1800/5037 conversations
  Indexed 1900/5037 conversations
  Indexed 2000/5037 conversations
  Indexed 2100/5037 conversations
  Indexed 2200/5037 conversations
  Indexed 2300/5037 conversations
  Indexed 2400/5037 conversations
  Indexed 2500/5037 conversations
  Indexed 2600/5037 conversations
  Indexed 2700/5037 conversations
  Indexed 2800/5037 conversations
  Indexed 2900/5037 conversations
  Indexed 3000/5037 con

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/pravaah_data_file/* /content/


In [ ]:
from chromadb.utils import embedding_functions

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="multi-qa-mpnet-base-dot-v1",
    device="cuda"
)

chroma_client = chromadb.PersistentClient(path="../Codebase/chroma_db_turns3")

collection = chroma_client.get_collection(
    name="evidence_turns",
    embedding_function=embedding_func
)
#
results = collection.query(
    query_texts=[user_query],
    n_results=9
)

In [ ]:
import chromadb
import pandas as pd
import json

# 1. LOAD DATASETS
turn_df = pd.read_csv("../Codebase/utterances_final.csv")
conv_summary_df = pd.read_csv("../Codebase/conversation_level_summary.csv")

# 2. CONNECT TO EVIDENCE LAYER (Turn-level)
# We query the turns database because it contains the 51k "causal" moments
chroma_client = chromadb.PersistentClient(path="./chroma_db_turns")
collection = chroma_client.get_collection(name="evidence_turns")

# 3. YOUR CAUSAL QUERY
user_query = "Why do customers escalate to supervisors?"

results = collection.query(
    query_texts=[user_query],
    n_results=9
)

# 4. EXTRACT DATA
retrieved_turn_ids = results['ids'][0]
retrieved_distances = results['distances'][0]
retrieved_metadatas = results['metadatas'][0]

print(f"🔍 QUERY: {user_query}")
print(f"✅ Found {len(retrieved_turn_ids)} specific evidence turns.\n")

# 5. PROCESS RESULTS
unique_transcripts_processed = set()

for i, turn_id in enumerate(retrieved_turn_ids):
    transcript_id = turn_id.split('_')[0]

    print(f"\n{'='*70}")
    print(f"EVIDENCE {i+1} | TURN ID: {turn_id}")
    print(f"Similarity Score: {1 - retrieved_distances[i]:.4f}")
    print(f"{'='*70}")

    # Lookup summary context
    try:
        conv_summary = conv_summary_df[conv_summary_df['transcript_id'] == transcript_id].iloc[0]

        print(f"Domain: {conv_summary['domain']}")
        print(f"Intent: {conv_summary['intent']}")
        print(f"Outcome: {conv_summary['outcome']}")
        print(f"Reason for Call: {conv_summary['reason_for_call']}")

        # Pull the specific turns for this transcript
        conv_turns = turn_df[turn_df['transcript_id'] == transcript_id].sort_values('turn_no')

        print(f"\nConversation Excerpt ({len(conv_turns)} turns):")
        print("-" * 70)

        # Print only a window around the retrieved turn for clarity
        target_turn_no = int(turn_id.split('_')[1])
        for _, turn in conv_turns.iterrows():
            # Highlight the turn that the AI actually found
            prefix = "👉 " if turn['turn_no'] == target_turn_no else "   "
            print(f"{prefix}Turn {turn['turn_no']} ({turn['speaker']}): {turn['text'][:150]}...")

    except IndexError:
        print(f"⚠️ Transcript {transcript_id} not found in summary dataframe.")

print(f"\n{'='*70}")
print("SUMMARY")
print(f"{'='*70}")
print(f"Retrieved Evidence IDs: {', '.join(retrieved_turn_ids)}")

InvalidArgumentError: Collection expecting embedding with dimension of 768, got 384

In [ ]:
import shutil
import os

paths_to_clear = ["./chroma_db_turns", "./chroma_db_conversations"]

for path in paths_to_clear:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"🗑️ Deleted: {path}")
    else:
        print(f"⚪ Path not found (already clean): {path}")

print("\n✨ Environment is now 100% clean.")

🗑️ Deleted: ./chroma_db_turns
⚪ Path not found (already clean): ./chroma_db_conversations

✨ Environment is now 100% clean.


In [ ]:
import chromadb
import pandas as pd
from chromadb.utils import embedding_functions

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="multi-qa-mpnet-base-dot-v1",
    device="cuda"
)

chroma_client = chromadb.PersistentClient(path="../Codebase/chroma_db_turns3")
collection = chroma_client.get_collection(
    name="evidence_turns",
    embedding_function=embedding_func
)

# 2. QUERY
user_query = "Why do customers escalate to supervisors?"

results = collection.query(
    query_texts=[user_query],
    n_results=9
)

retrieved_turn_ids = results['ids'][0]
retrieved_distances = results['distances'][0]
retrieved_metadatas = results['metadatas'][0]

print(f"Query: {user_query}")
print(f"Retrieved {len(retrieved_turn_ids)} causal evidence points.")

for i, full_id in enumerate(retrieved_turn_ids):
    transcript_id, turn_no = full_id.split('_')
    turn_no = int(turn_no)

    print(f"\n{'='*70}")
    print(f"EVIDENCE {i+1}: Transcript {transcript_id} | Turn {turn_no}")
    print(f"Similarity Score: {1 - retrieved_distances[i]:.4f}")
    print(f"{'='*70}")

    conv_summary = conv_summary_df[conv_summary_df['transcript_id'] == transcript_id].iloc[0]

    print(f"Domain: {conv_summary['domain']}")
    print(f"Outcome: {conv_summary['outcome']}")
    print(f"Context: {conv_summary['reason_for_call']}")

    conv_turns = turn_df[turn_df['transcript_id'] == transcript_id].sort_values('turn_no')

    print(f"\nDialogue Excerpt (Focusing on Turn {turn_no}):")
    print("-" * 70)

    window = conv_turns[(conv_turns['turn_no'] >= turn_no - 2) & (conv_turns['turn_no'] <= turn_no + 2)]

    for _, turn in window.iterrows():
        marker = "👉" if turn['turn_no'] == turn_no else "  "
        print(f"{marker} Turn {turn['turn_no']} ({turn['speaker']}): {turn['text'][:150]}...")
        if turn['turn_no'] == turn_no:
            print(f"   [Sentiment: {turn['sentiment']:.3f} | Metadata Patterns: {retrieved_metadatas[i].get('patterns')}]")

print(f"\n{'='*70}")
print("RETRIEVAL SUMMARY")
print(f"{'='*70}")
print(f"Found evidence in transcripts: {', '.join(list(set([tid.split('_')[0] for tid in retrieved_turn_ids])))}")

Query: Why do customers escalate to supervisors?
Retrieved 9 causal evidence points.

EVIDENCE 1: Transcript 4296-2682-1344-3926 | Turn 16
Similarity Score: 26.6229
Domain: telecommunications
Outcome: escalation
Context: customer angela fisher escalated to supervisor after experiencing multiple failed card activations and poor service recovery attempts over three weeks.

Dialogue Excerpt (Focusing on Turn 16):
----------------------------------------------------------------------
   Turn 14 (customer): why wasn't i notified about this? and why didn't the other agents see this?...
   Turn 15 (agent): i sincerely apologize. this should have been communicated to you. i can verify your activation system right now, which will resolve the issue....
👉 Turn 16 (customer): i still want to speak with a supervisor. this has been incredibly frustrating and i want to file a formal complaint....
   [Sentiment: -0.954 | Metadata Patterns: ['anger_expression', 'complaint_intent', 'unmet_expectations',

In [23]:
import re
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from google import genai
from google.colab import userdata
import json
import os

turn_df = pd.read_csv("../Codebase/utterances_final.csv")
conv_summary_df = pd.read_csv("../Codebase/conversation_level_summary.csv")

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="multi-qa-mpnet-base-dot-v1",
    device="cuda"
)

chroma_client = chromadb.PersistentClient(path="../Codebase/chroma_db_turns3")
evidence_collection = chroma_client.get_collection(
    name="evidence_turns",
    embedding_function=embedding_func
)

class CausalQuerySystem:
    def __init__(self, turn_df, conv_summary_df, collection, api_key):
        self.turn_df = turn_df
        self.conv_summary_df = conv_summary_df
        self.collection = collection
        self.client = genai.Client(api_key=api_key)

    def _expand_query_intent(self, user_query):


        prompt = f"""
        You are an expert search optimizer.
        The user is asking: "{user_query}"

        Write a hypothetical customer service dialogue turn (1-2 sentences) that would appear in a transcript related to this issue.
        Include keywords like 'frustrated', 'supervisor', 'delivery', 'fraud', or specific error codes if relevant.
        Focus on the root cause.
        """
        try:
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            expanded_text = response.text.strip()
            print(f"Query Expanded to: {expanded_text[:100]}...")
            return expanded_text
        except:
            return user_query

    def _get_category(self, user_query):



        prompt = f"""
        Classify this customer service query into ONE of these categories:
        - Escalation Analysis
        - Fraud & Security
        - Logistics & Delivery
        - Billing & Payments
        - General Support

        QUERY: "{user_query}"

        Output only the category name.
        """
        try:
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            return response.text.strip()
        except:
            return "Causal Analysis"

    def query(self, user_query: str, top_k: int = 9):
        print(f"\n{'='*70}")
        print(f"🔍 CAUSAL QUERY: {user_query}")
        print(f"{'='*70}")

        search_query = self._expand_query_intent(user_query)

        results = self.collection.query(
            query_texts=[search_query],
            n_results=top_k
        )

        retrieved_ids = results['ids'][0]
        retrieved_metas = results['metadatas'][0]
        retrieved_distances = results['distances'][0]

        print(f"✓ Found {len(retrieved_ids)} specific evidence points.")

        evidence_list = []
        unique_transcripts = set()

        for i, full_id in enumerate(retrieved_ids):
            tid, turn_no = full_id.split('_')
            turn_no = int(turn_no)
            unique_transcripts.add(tid)

            meta = retrieved_metas[i]

            try:
                turn_row = self.turn_df[
                    (self.turn_df['transcript_id'] == tid) &
                    (self.turn_df['turn_no'] == turn_no)
                ].iloc[0]
                text_content = turn_row['text']
                speaker = turn_row['speaker']
            except:
                text_content = "Text not found"
                speaker = "Unknown"

            evidence_list.append({
                'transcript_id': tid,
                'turn_no': turn_no,
                'speaker': speaker,
                'text': text_content,
                'outcome': meta.get('outcome', 'Unknown'),
                'sentiment': meta.get('sentiment', 0.0),
                'patterns': meta.get('patterns', '[]'),
                'similarity': round(1 - retrieved_distances[i], 4)
            })

        context_docs = []
        for tid in list(unique_transcripts)[:3]:
            try:
                summary = self.conv_summary_df[self.conv_summary_df['transcript_id'] == tid].iloc[0]


                turns = self.turn_df[self.turn_df['transcript_id'] == tid].sort_values('turn_no')
                turn_text = "\n".join([f"T{t['turn_no']} ({t['speaker']}): {t['text'][:100]}" for _, t in turns.head(20).iterrows()])

                context_docs.append(f"""
                TRANSCRIPT {tid}:
                - Outcome: {summary['outcome']}
                - Reason: {summary['reason_for_call']}
                - Excerpt:
                {turn_text}
                """)
            except:
                continue

        # 4. Generate Causal Explanation
        print(f"✓ Generating explanation with Gemini...")
        explanation = self._generate_gemini_response(user_query, evidence_list, context_docs)

        return {
            'query': user_query,
            'explanation': explanation,
            'evidence_count': len(evidence_list),
            'retrieved_ids': list(unique_transcripts)
        }

    def _generate_gemini_response(self, query, evidence, context):
        # Format the evidence list for the prompt
        evidence_str = ""
        for i, ev in enumerate(evidence, 1):
            evidence_str += f"""
            {i}. Transcript {ev['transcript_id']} (Turn {ev['turn_no']})
               - Speaker: {ev['speaker']}
               - Text: "{ev['text']}"
               - Sentiment: {ev['sentiment']}
               - Detected Patterns: {ev['patterns']}
            """

        context_str = "\n".join(context)

        prompt = f"""
You are an expert Causal Analysis AI. Explain WHY specific outcomes occurred using the provided evidence.

USER QUERY: "{query}"

RETRIEVED CAUSAL EVIDENCE:
{evidence_str}

CONVERSATION CONTEXT:
{context_str}

INSTRUCTIONS:
1. **ULTRA-BREVITY:** Total response MUST be under 100 words. Use basic, simple English.
2. **STRICT QUANTITY:** You MUST provide EXACTLY 4 Causal Factors and EXACTLY 4 Recommendations.
3. **TELEGRAPHIC STYLE:** Use very short phrases (3-5 words) for bullets. No long sentences.
4. **CITATIONS:** Every single bullet MUST end with: ***(Source: Transcript ID, Turn No)***.

FORMAT:
## Causal Explanation
[1-sentence summary].

## Key Causal Factors
- [Short Phrase] ***(Source: ID, Turn)***
- [Short Phrase] ***(Source: ID, Turn)***
- [Short Phrase] ***(Source: ID, Turn)***
- [Short Phrase] ***(Source: ID, Turn)***

## Recommendations
1. **[Short Name]:** [Action] ***(Source: ID, Turn)***
2. **[Short Name]:** [Action] ***(Source: ID, Turn)***
3. **[Short Name]:** [Action] ***(Source: ID, Turn)***
4. **[Short Name]:** [Action] ***(Source: ID, Turn)***

[METADATA_START]
GROUNDING: [1 sentence on which pattern was most important]
CONFIDENCE: [High/Medium based on data match]
[METADATA_END]
"""

        try:
            response = self.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            return response.text
        except Exception as e:
            return f"Error generating explanation: {e}"

try:
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = input("Enter Gemini API Key: ")

# Initialize System
system = CausalQuerySystem(
    turn_df=turn_df,
    conv_summary_df=conv_summary_df,
    collection=evidence_collection,
    api_key=API_KEY
)

TEST_QUERIES = [
    "Why do customers escalate to supervisors?",
    "What triggers fraud alert investigations?",
    "Why do customers complain about delivery status?",
    "What causes customers to call back multiple times for the same issue?",
    "Why are refund requests frequently delayed or denied?",
    "What agent behaviors lead to negative customer sentiment?",
    "What technical issues cause login or transaction failures?",
    "Why do identity verification procedures cause customer frustration?",
    "What patterns lead to customers threatening to close their accounts?",
    "What are the primary triggers for sudden spikes in customer anger?"
]

all_results = []

# Run Loop
print("\n" + "="*70)
print("STARTING BATCH PROCESSING")
print("="*70)


for i, q in enumerate(TEST_QUERIES):
    result = system.query(q)

    full_text = result['explanation']

    dynamic_category = system._get_category(q)

    metadata_match = re.search(r"\[METADATA_START\](.*?)\[METADATA_END\]", full_text, re.DOTALL)
    gemini_meta = metadata_match.group(1).strip() if metadata_match else "General analysis."

    clean_output = re.sub(r"\[METADATA_START\].*?\[METADATA_END\]", "", full_text, flags=re.DOTALL).strip()

    remarks_str = (
        f"🔍 {gemini_meta} | "
        f"Model: multi-qa-mpnet-base (768-dim) | "
        f"Evidence: {result['evidence_count']} turns from {len(result['retrieved_ids'])} transcripts | "
        f"Data Floor: -0.954 Sentiment."
    )

    all_results.append({
        "Query_Id": f"Q{i+9:03d}",
        "Query": q,
        "Query_Category": dynamic_category,
        "System_Output": clean_output,
        "Remarks": remarks_str
    })

    print("\n💡 SYSTEM OUTPUT PREVIEW:")
    print(result['explanation'][:500] + "...\n")

# Save to CSV
submission_df = pd.DataFrame(all_results)
submission_df.to_csv("task1_queries.csv", index=False)
print("✅ Saved 'task1_queries.csv' successfully!")


STARTING BATCH PROCESSING

🔍 CAUSAL QUERY: Why do customers escalate to supervisors?
  🧠 Query Expanded to: "I'm extremely **frustrated** because this ongoing **delivery** issue has caused significant financi...
✓ Found 9 specific evidence points.
✓ Generating explanation with Gemini...

💡 SYSTEM OUTPUT PREVIEW:
## Causal Explanation
Customers escalate due to unresolved recurring issues and a belief that the current representative cannot provide an adequate solution or compensation.

## Key Causal Factors
- Repeated missing items issues ***(Source: 4486-6634-9566-4432, T2)***
- Perception of agents saying "same things" ***(Source: 4486-6634-9566-4432, T8)***
- Past unfulfilled promises of fix ***(Source: 4486-6634-9566-4432, T6)***
- Demand for manager/full compensation ***(Source: 4486-6634-9566-4432, ...

✅ Saved 'task1_queries.csv' successfully!


In [ ]:
print("Hello")

Hello


In [24]:
def get_full_transcript_text(transcript_ids, turn_df):
    """
    Pulls the complete dialogue history for specific transcript IDs.
    Used for Deep-Dive reasoning in Turn 2+.
    """
    context_text = ""
    for tid in transcript_ids:
        # Get all turns for this transcript, sorted
        chat_rows = turn_df[turn_df['transcript_id'] == tid].sort_values('turn_no')

        context_text += f"\n\n--- TRANSCRIPT {tid} ---\n"
        for _, row in chat_rows.iterrows():
            context_text += f"T{row['turn_no']} ({row['speaker']}): {row['text']}\n"

    return context_text

In [37]:
from google.colab import userdata
from google import genai
import pandas as pd
import time

class RobustMultiTurnSystem:
    def __init__(self, task1_engine):
        self.engine = task1_engine
        self.sessions = {}

        try:

            secret_key = userdata.get('GEMINI_API_KEY')
            if secret_key:
                print("API KEY")
                self.engine.client = genai.Client(api_key=secret_key)
            else:
                print("No APi key")
        except Exception as e:
            print(f"No api key")

    def start_session(self, session_id):
        self.sessions[session_id] = {
            'history': [],
            'evidence_lock': [],
            'full_context_cache': ""
        }
        print(f"Session {session_id}")



    def _get_category(self, user_query):
        prompt = f"""
        Classify this customer service query into ONE of these categories:
        - Escalation Analysis
        - Fraud & Security
        - Logistics & Delivery
        - Billing & Payments
        - General Support

        QUERY: "{user_query}"

        Output only the category name.
        """
        try:
            response = self.engine.client.models.generate_content(
                model='gemini-2.5-flash',
                contents=prompt
            )
            return response.text.strip()
        except:
            return "Causal Analysis"


    def query(self, user_query, session_id, turn_number):
        session = self.sessions[session_id]

        if turn_number == 1:
            print(f"Turn 1 Retrieval for: {user_query}")

            results = self.engine.query(user_query, top_k=5)

            found_ids = results['retrieved_ids']
            session['evidence_lock'] = found_ids

            session['full_context_cache'] = get_full_transcript_text(found_ids, self.engine.turn_df)

            response = results['explanation']

        else:
            print(f"Turn {turn_number} Reasoning (Locked to {len(session['evidence_lock'])} transcripts)")

            history_str = ""
            for turn in session['history']:
                history_str += f"User: {turn['query']}\nAI: {turn['response']}\n"

            prompt = f"""
            You are an expert analyst having a conversation about specific customer service transcripts.

            LOCKED EVIDENCE (Do not discuss other transcripts):
            {session['full_context_cache']}

            CONVERSATION HISTORY:
            {history_str}

            CURRENT QUESTION: "{user_query}"

            INSTRUCTIONS:
            1. Answer the question specifically using the LOCKED EVIDENCE provided above.
            2. Cite specific Turn Numbers (e.g., "In T12...") to prove your point.
            3. If the user asks for a comparison, compare the locked transcripts.
            4. Keep the response under 200 words.
            5. Inside the Reasoning you can add (Source: ID, Turn) there is 2 to 3 things must required
            6. Inside Evidence there is 5 to 6 minimum point which support the reasoing
            7. If you thing the (Source: ID, Turn) have nothing work don't add that
            8. ONLY REQUIRED THINK WHICH SATASIFY THE QUERY ONELY THAT THING YOU HAVE TO SUPPLY
            9. ***THE RULE IS STREET WHEN YOU WRITE (Source: ID, Turn) MAKE SURE ID IS 'XXXX-XXXX-XXXX-XXXX'***

            Output Format:
            ## Reasoning
            [Answer]
            ## Evidence
            - [Point 1] (Source: ID, Turn)
            - [Point 2] (Source: ID, Turn)
            """

            try:
                response = self.engine.client.models.generate_content(
                    model='gemini-2.5-flash',
                    contents=prompt
                ).text
            except Exception as e:
                response = f"Error in reasoning: {e}"

        session['history'].append({
            'query': user_query,
            'response': response
        })

        return response

In [47]:
import time

secret_key = userdata.get('GEMINI_API_KEY')
if secret_key:
    print("API KEY")
else:
    print("No API KEY")

system = CausalQuerySystem(
    turn_df=turn_df,
    conv_summary_df=conv_summary_df,
    collection=evidence_collection,
    api_key=secret_key
)

task2_system = RobustMultiTurnSystem(system)

# 2. Define the Test Chains
TEST_CHAINS = [
    {
        "session_id": "S001",
        "topic": "Escalation",
        "turns": [
            "Why do customers escalate to supervisors?",
            "Did the agent try to de-escalate before the manager was requested?",
            "What was the specific sentiment score when the customer got angry?"
        ]
    },
    {
        "session_id": "S006",
        "turns": [
            "What product features did customers ask about most?",
            "Did agents provide accurate and complete product information?",
            "Were there any upselling or cross-selling attempts?"
         ]
    }
]

# 3 - Execution Loop
task2_results = []

for chain in TEST_CHAINS:
    sid = chain['session_id']
    task2_system.start_session(sid)

    for i, q in enumerate(chain['turns'], 1):
        print(f"\nProcessing {sid} - Turn {i}...")

        # Query
        output = task2_system.query(q, sid, turn_number=i)
        dynamic_Query_Category = task2_system._get_category(q)

        # Save Result
        task2_results.append({
            "Query_Id": f"Q{sid}_{chr(64 + i)}",
            "Query_Category": dynamic_Query_Category,
            "Query": q,
            "System_Output": output,
            "Remarks": str(task2_system.sessions[sid]['evidence_lock'])
        })

        time.sleep(20)

pd.DataFrame(task2_results).to_csv("task60_interactive_submission.csv", index=False)

🔐 Successfully loaded 'GEMINI_API_KEY' from Google Secrets.
🔐 Successfully loaded 'GEMINI_API_KEY' from Google Secrets.

🚀 STARTING TASK 2: INTERACTIVE SESSIONS
🟢 Session S006 Started

Processing S006 - Turn 1...
  🔍 Turn 1 Retrieval for: What product features did customers ask about most?

🔍 CAUSAL QUERY: What product features did customers ask about most?
  🧠 Query Expanded to: "I'm completely frustrated because the 'auto-sync' feature isn't working at all, consistently displa...
✓ Found 5 specific evidence points.
✓ Generating explanation with Gemini...

Processing S006 - Turn 2...
  🧠 Turn 2 Reasoning (Locked to 5 transcripts)

Processing S006 - Turn 3...
  🧠 Turn 3 Reasoning (Locked to 5 transcripts)

✅ Task 2 Complete! Saved to 'task2_interactive_submission.csv'


In [48]:
dataset6 = pd.read_csv("task60_interactive_submission.csv")
dataset5 = pd.read_csv("task50_interactive_submission.csv")
dataset4 = pd.read_csv("task40_interactive_submission.csv")
dataset3 = pd.read_csv("task30_interactive_submission.csv")
dataset2 = pd.read_csv("task20_interactive_submission.csv")
dataset1 = pd.read_csv("task10_interactive_submission.csv")
dataset7 = pd.read_csv("task70_interactive_submission.csv")

In [49]:
final_dataset = pd.concat([
    dataset1,
    dataset2,
    dataset3,
    dataset4,
    dataset5,
    dataset6,
    dataset7
], ignore_index=True)

In [50]:
final_dataset

,Query_Id,Query_Category,Query,System_Output,Remarks
0,QS002_A,Fraud & Security,What triggers fraud alert investigations?,## Causal Explanation\nInvestigations are trig...,"['7990-9814-9440-1658', '1058-9135-5810-9284',..."
1,QS002_B,Fraud & Security,Did the customer verify their identity success...,"## Reasoning\nYes, in all provided transcripts...","['7990-9814-9440-1658', '1058-9135-5810-9284',..."
2,QS002_C,Escalation Analysis,Compare the agent's helpfulness in the first v...,## Reasoning\nThe agents in both the first and...,"['7990-9814-9440-1658', '1058-9135-5810-9284',..."
3,QS001_A,Causal Analysis,Why do customers escalate to supervisors?,## Causal Explanation\nCustomers escalate to s...,"['9348-3845-9721-4391', '9391-7266-1346-7002',..."
4,QS001_B,Causal Analysis,Did the agent try to de-escalate before the ma...,"## Reasoning\nNo, the agent did not attempt to...","['9348-3845-9721-4391', '9391-7266-1346-7002',..."
5,QS001_C,Causal Analysis,What was the specific sentiment score when the...,## Reasoning\nThe provided transcripts do not ...,"['9348-3845-9721-4391', '9391-7266-1346-7002',..."
6,QS003_A,Billing & Payments,What are the most common reasons customers req...,## Causal Explanation\nCustomers request refun...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."
7,QS003_B,Billing & Payments,How quickly did agents process refund requests?,## Reasoning\nAgents documented the provision ...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."
8,QS003_C,Billing & Payments,Was the refund policy clearly explained to cus...,## Reasoning\nBased on the provided transcript...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."
9,QS003_D,Billing & Payments,Did customers seem satisfied with the refund r...,## Reasoning\nCustomers did not seem entirely ...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."


In [52]:
df_sorted = final_dataset.sort_values(by='Query_Id', ascending=True)

In [53]:
df_sorted

,Query_Id,Query_Category,Query,System_Output,Remarks
3,QS001_A,Causal Analysis,Why do customers escalate to supervisors?,## Causal Explanation\nCustomers escalate to s...,"['9348-3845-9721-4391', '9391-7266-1346-7002',..."
4,QS001_B,Causal Analysis,Did the agent try to de-escalate before the ma...,"## Reasoning\nNo, the agent did not attempt to...","['9348-3845-9721-4391', '9391-7266-1346-7002',..."
5,QS001_C,Causal Analysis,What was the specific sentiment score when the...,## Reasoning\nThe provided transcripts do not ...,"['9348-3845-9721-4391', '9391-7266-1346-7002',..."
0,QS002_A,Fraud & Security,What triggers fraud alert investigations?,## Causal Explanation\nInvestigations are trig...,"['7990-9814-9440-1658', '1058-9135-5810-9284',..."
1,QS002_B,Fraud & Security,Did the customer verify their identity success...,"## Reasoning\nYes, in all provided transcripts...","['7990-9814-9440-1658', '1058-9135-5810-9284',..."
2,QS002_C,Escalation Analysis,Compare the agent's helpfulness in the first v...,## Reasoning\nThe agents in both the first and...,"['7990-9814-9440-1658', '1058-9135-5810-9284',..."
6,QS003_A,Billing & Payments,What are the most common reasons customers req...,## Causal Explanation\nCustomers request refun...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."
7,QS003_B,Billing & Payments,How quickly did agents process refund requests?,## Reasoning\nAgents documented the provision ...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."
8,QS003_C,Billing & Payments,Was the refund policy clearly explained to cus...,## Reasoning\nBased on the provided transcript...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."
9,QS003_D,Billing & Payments,Did customers seem satisfied with the refund r...,## Reasoning\nCustomers did not seem entirely ...,"['1849-7167-6629-2544', '4060-3523-1704-8411',..."


In [54]:
df_sorted.to_csv("Task_2.csv", index=False)


Sorted dataset saved as 'sorted_submission.csv'


In [55]:
###END